In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option("max_columns", 100)
pd.set_option('display.max_rows',500)
from matplotlib.pyplot import figure as fg
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("/content/drive/My Drive/data.csv")
df.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_opp_KD,B_avg_opp_LEG_att,...,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_att,R_avg_TD_landed,R_avg_TD_pct,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_longest_win_streak,R_losses,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_opp_KD,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_opp_PASS,R_avg_opp_REV,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_opp_SIG_STR_pct,R_avg_opp_SUB_ATT,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_opp_TD_pct,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_total_rounds_fought,R_total_time_fought(seconds),R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019-06-08,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0.0,4.0,0.0,9.200000,6.000000,0.200000,0.000000,62.600000,20.600000,2.600000,2.000000,48.600000,11.200000,0.800000,7.6,5.400000,0.400000,0.000000,65.40,22.600000,0.466000,0.400000,0.80000,0.200000,0.100000,66.400000,23.600000,4.0,1.0,6.400000,4.000000,1.000000,0.60000,51.200000,17.400000,0.600000,0.200000,39.600000,9.400000,0.200000,6.80000,...,44.000000,0.466000,0.100000,5.300000,1.900000,0.458000,129.900000,69.100000,4.0,2.0,13.300000,8.800000,7.500000,5.100000,90.500000,26.800000,0.800000,0.300000,76.100000,17.300000,0.100000,9.400000,6.100000,0.000000,0.000000,98.800000,32.200000,0.336000,0.000000,0.900000,0.100000,0.050000,110.500000,43.300000,27.0,742.60,3.0,0.0,2.0,4.0,2.0,0.0,0.0,8.0,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019-06-08,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0.0,3.0,0.0,14.600000,9.100000,11.800000,7.300000,124.700000,42.100000,2.400000,1.900000,112.000000,32.000000,0.000000,12.3,10.200000,0.800000,0.000000,138.90,51.300000,0.399000,0.700000,1.00000,0.500000,0.225000,158.700000,69.600000,3.0,6.0,13.000000,9.300000,12.800000,9.60000,101.700000,32.000000,8.100000,6.900000,97.700000,30.800000,0.100000,11.90000,...,59.428571,0.575714,0.428571,5.142857,2.428571,0.601429,161.571429,102.857143,2.0,2.0,24.571429,14.142857,10.571429,7.857143,98.571429,32.571429,6.428571,4.285714,61.857143,12.428571,0.000000,29.142857,18.142857,1.142857,0.000000,115.571429,44.714286,0.437143,0.285714,3.285714,0.857143,0.147143,158.142857,82.285714,25.0,1062.00,2.0,0.0,1.0,2.0,0.0,2.0,0.0,5.0,Southpaw,165.10,167.64,125.0,32.0,31.0
2,Tony Ferguson,Donald Cerrone,Dan Miragliotta,2019-06-08,"Chicago, Illinois, USA",Red,False,Lightweight,3,0.0,3.0,0.0,15.354839,11.322581,6.741935,4.387097,84.741935,38.580645,5.516129,3.806452,67.645161,23.258065,0.645161,14.0,12.193548,0.935484,0.096774,97.00,46.774194,0.496129,0.354839,2.16129,0.677419,0.295484,103.709677,52.548387,8.0,8.0,17.903226,11.870968,8.419355,5.83871,84.548387,38.064516,1.741935,0.935484,67.645161,25.483871,0.225806,9.16129,...,55.466667,0.430000,1.000000,0.933333,0.400000,0.277333,1

# **Data Cleaning**
based on the output of pandas profiling we can see lot of missing values and highly correlated variables, we need to get rid of them.

In [ ]:
# We see that avg_body_att has the most no. of missing values and also it is highly correlated to avg_body_landed, so its of no use
df=df.dropna(subset=['B_avg_BODY_att'])
df=df.dropna(subset=['R_avg_BODY_att'])

In [ ]:
# Referee is of no use for the analysis, and draws has constant 0's
df.drop(['Referee','B_draw','R_draw'], axis=1, inplace=True)
#filling rest of the null values with mean
df.fillna(df.mean(), inplace= True)

In [ ]:
#all the attempt variables are highly correlated with landed variables, hence dropped 
df=df.drop([col for col in df.columns if 'att' in col],axis=1)
df=df.drop(['B_total_rounds_fought','R_total_rounds_fought'],axis=1)
df.shape

(3648, 104)

In [ ]:
#Missing values in type of  stance are unknown, so putting them in others along with stance which has less frequency
df['B_Stance']=df['B_Stance'].apply(lambda x : 'Other' if x not in ['Orthodox','Southpaw'] else x )
df['R_Stance']=df['R_Stance'].apply(lambda x : 'Other' if x not in ['Orthodox','Southpaw'] else x )

In [ ]:
df.drop(['R_fighter','B_fighter','date','location'],axis=1,inplace=True)
df=df[df['Winner']!='Draw']
df['title_bout']=df['title_bout'].apply(lambda x: 1 if 'True' else 0)
df['Winner']=df['Winner'].apply(lambda x: 1 if x == 'Red' else 0)
#One Hot encoding
df=pd.get_dummies(df)
df.head()

,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_avg_BODY_landed,B_avg_CLINCH_landed,B_avg_DISTANCE_landed,B_avg_GROUND_landed,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_landed,B_avg_opp_HEAD_landed,B_avg_opp_KD,B_avg_opp_LEG_landed,B_avg_opp_PASS,B_avg_opp_REV,B_avg_opp_SIG_STR_landed,B_avg_opp_SIG_STR_pct,B_avg_opp_SUB_ATT,B_avg_opp_TD_landed,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_landed,B_total_time_fought(seconds),B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,...,R_losses,R_avg_opp_BODY_landed,R_avg_opp_CLINCH_landed,R_avg_opp_DISTANCE_landed,R_avg_opp_GROUND_landed,R_avg_opp_HEAD_landed,R_avg_opp_KD,R_avg_opp_LEG_landed,R_avg_opp_PASS,R_avg_opp_REV,R_avg_opp_SIG_STR_landed,R_avg_opp_SIG_STR_pct,R_avg_opp_SUB_ATT,R_avg_opp_TD_landed,R_avg_opp_TD_pct,R_avg_opp_TOTAL_STR_landed,R_total_time_fought(seconds),R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,weight_class_Women's Bantamweight,weight_class_Women's Featherweight,weight_class_Women's Flyweight,weight_class_Women's Strawweight,B_Stance_Orthodox,B_Stance_Other,B_Stance_Southpaw,R_Stance_Orthodox,R_Stance_Other,R_Stance_Southpaw
0,1,1,5,0.0,4.0,6.000000,0.000000,20.600000,2.000000,11.200000,0.800000,5.400000,0.400000,0.000000,22.600000,0.466000,0.400000,0.200000,0.100000,23.600000,4.0,1.0,4.000000,0.60000,17.400000,0.200000,9.400000,0.200000,4.800000,0.000000,0.000000,18.20000,0.236000,0.000000,0.400000,0.100000,19.200000,419.400000,0.0,0.0,1.0,0.0,2.0,1.0,0.0,4.0,167.64,170.18,135.0,0.0,...,2.0,8.800000,5.100000,26.800000,0.300000,17.300000,0.100000,6.100000,0.000000,0.000000,32.200000,0.336000,0.000000,0.100000,0.050000,43.300000,742.60,3.0,0.0,2.0,4.0,2.0,0.0,0.0,8.0,162.56,162.56,135.0,31.0,32.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,1,1,5,0.0,3.0,9.100000,7.300000,42.100000,1.900000,32.000000,0.000000,10.200000,0.800000,0.000000,51.300000,0.399000,0.700000,0.500000,0.225000,69.600000,3.0,6.0,9.300000,9.60000,32.000000,6.900000,30.800000,0.100000,8.400000,1.400000,0.000000,48.50000,0.408000,0.700000,0.900000,0.231000,75.400000,849.000000,0.0,0.0,2.0,1.0,0.0,0.0,1.0,4.0,167.64,167.64,125.0,0.0,...,2.0,14.142857,7.857143,32.571429,4.285714,12.428571,0.000000,18.142857,1.142857,0.000000,44.714286,0.437143,0.285714,0.857143,0.147143,82.285714,1062.00,2.0,0.0,1.0,2.0,0.0,2.0,0.0,5.0,165.10,167.64,125.0,32.0,31.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
2,1,1,3,0.0,3.0,11.322581,4.387097,38.580645,3.806452,23.258065,0.645161,12.193548,0.935484,0.096774,46.774194,0.496129,0.354839,0.677419,0.295484,52.548387,8.0,8.0,11.870968,5.83871,38.064516,0.935484,25.483871,0.225806,7.483871,0.032258,0.032258,44.83871,0.453226,0.096774,0.225806,0.063548,49.774194,581.870968,1.0,0.0,0.0,7.0,10.0,6.0,0.0,23.0,185.42,185.42,155.0,0.0,...,1.0,8.133333,0.733333,32.200000,2.800000,23.200000,0.266667,4.400000,0.333333,0.133333,35.733333,0.340000,0.066667,0.666667,0.131333,38.600000,604.40,2.0,0.0,1.0,3.0,3.0,6.0,1.0,14.0,180.34,193.04,155.0,36.0,35.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
3,0,1,3,0.0,4.0,14.000000,11.000000,48.750000,10.500000,53.750000,0.500000,2.500000,0.500000,0.250000,70.250000,0.550000,0.250000

In [ ]:
x=df[df.columns[df.columns!='Winner']]
y=df['Winner']

In [ ]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 0)

In [ ]:
scaler=StandardScaler()
xTrain= scaler.fit_transform(xTrain)
xTest= scaler.transform(xTest)

In [ ]:
print(xTrain.shape, xTest.shape)

(2873, 116) (719, 116)


In [ ]:
model_scores=[]
#Logistic Regression
param={'C':[0.1,1,10],'penalty':['l1','l2']}
lg_clf= GridSearchCV(LogisticRegression(),param, cv=5).fit(xTrain,yTrain)
model_scores.append({'Model':'LogisticRegression','Train Score':lg_clf.best_score_,'Test Score':lg_clf.score(xTest,yTest),'Parameters':lg_clf.best_params_})

#Decision Tree 
param={'max_depth': np.arange(3, 10)}
dt_clf=GridSearchCV(DecisionTreeClassifier(), param,cv=5).fit(xTrain,yTrain)
model_scores.append({'Model':'Decision Tree','Train Score':dt_clf.best_score_,'Test Score':dt_clf.score(xTest,yTest),'Parameters':dt_clf.best_params_})

#KNN
param= {'n_neighbors':list(range(1,16))}
knn_clf= GridSearchCV(KNeighborsClassifier(),param,cv=5).fit(xTrain,yTrain)
model_scores.append({'Model':'KNN','Train Score':knn_clf.best_score_,'Test Score':knn_clf.score(xTest,yTest),'Parameters':knn_clf.best_params_})

#Support Vector machine
Cs = [0.01, 0.1, 1, 10]
gammas = [0.01, 0.1, 1]
param_grid = {'C': Cs,'gamma':gammas}
#svc rbf
svcr_clf = GridSearchCV(SVC(kernel='rbf',probability=True), param_grid, cv=3).fit(xTrain,yTrain)
model_scores.append({'Model':'SVC rbf','Train Score':svcr_clf.best_score_,'Test Score':svcr_clf.score(xTest,yTest),'Parameters':svcr_clf.best_params_})
#svc sigmoid
svcs_clf = GridSearchCV(SVC(kernel='sigmoid',probability=True), param_grid, cv=3).fit(xTrain,yTrain)
model_scores.append({'Model':'SVC sigmoid','Train Score':svcs_clf.best_score_,'Test Score':svcs_clf.score(xTest,yTest),'Parameters':svcs_clf.best_params_})
#svc poly
svcp_clf = GridSearchCV(SVC(kernel='poly',probability=True), param_grid, cv=3).fit(xTrain,yTrain)
model_scores.append({'Model':'SVC poly','Train Score':svcp_clf.best_score_,'Test Score':svcp_clf.score(xTest,yTest),'Parameters':svcp_clf.best_params_})
#Linear SVC
svcl_clf=GridSearchCV(SVC(kernel='linear',probability=True),param_grid={'C':Cs},cv=3).fit(xTrain,yTrain)
model_scores.append({'Model':'SVC Linear','Train Score':svcl_clf.best_score_,'Test Score':svcl_clf.score(xTest,yTest),'Parameters':svcl_clf.best_params_})

#Random Forrest
param_grid = {
    'bootstrap': [True],
    'max_depth': [4,16,64],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [200, 300, 1000]
}
rf_clf = GridSearchCV(RandomForestClassifier(), param_grid = param_grid, n_jobs = -1, verbose = 2).fit(xTrain,yTrain)
model_scores.append({'Model':'Random Forrest','Train Score':rf_clf.best_score_,'Test Score':rf_clf.score(xTest,yTest),'Parameters':rf_clf.best_params_})
project_1= pd.DataFrame(model_scores)
project_1

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   57.2s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:  9.7min finished


,Model,Train Score,Test Score,Parameters
0,LogisticRegression,0.683258,0.687065,"{'C': 0.1, 'penalty': 'l1'}"
1,Decision Tree,0.653324,0.632823,{'max_depth': 3}
2,KNN,0.654020,0.621697,{'n_neighbors': 9}
3,SVC rbf,0.678733,0.657858,"{'C': 1, 'gamma': 0.01}"
4,SVC sigmoid,0.671772,0.630042,"{'C': 0.01, 'gamma': 1}"
5,SVC poly,0.670031,0.632823,"{'C': 0.01, 'gamma': 0.01}"
6,SVC Linear,0.682214,0.667594,{'C': 10}
7,Random Forrest,0.687435,0.655076,"{'bootstrap': True, 'max_depth': 16, 'min_samp..."


In [ ]:
lg=LogisticRegression(C=0.1,penalty='l1')
dt=DecisionTreeClassifier(max_depth=3)
knn=KNeighborsClassifier(n_neighbors=9)
svcr=SVC(kernel='rbf',C=1,gamma=0.01,probability=True)
svcs=SVC(kernel='sigmoid',C=0.01,gamma=1,probability=True)
svcp=SVC(kernel='poly',C=0.01,gamma=0.01,probability=True)
svcl=SVC(kernel='linear',C=10,probability=True)
rf= RandomForestClassifier(bootstrap= True,max_depth=16,min_samples_leaf= 4,min_samples_split= 8,n_estimators= 300)

In [ ]:
#Voting Classifier
project_2=[]
estimators=[('knn',knn ), ('svc', svcs), ('log_reg', lg)]
ensemble_h= VotingClassifier(estimators, 'hard').fit(xTrain,yTrain)
project_2.append({'Model':'Ensembel hard','Train Score':ensemble_h.score(xTrain,yTrain),'Test Score':ensemble_h.score(xTest,yTest)})
estimators=[('svc',svcr ), ('log_reg', lg), ('rf', rf)]
ensemble_s= VotingClassifier(estimators, 'soft').fit(xTrain,yTrain)
project_2.append({'Model':'Ensembel soft','Train Score':ensemble_s.score(xTrain,yTrain),'Test Score':ensemble_s.score(xTest,yTest)})
voting=pd.DataFrame(project_2)
voting

,Model,Train Score,Test Score
0,Ensembel hard,0.701357,0.655076
1,Ensembel soft,0.846850,0.678720


In [ ]:
#Bagging
bag_lg_clf = BaggingClassifier(lg, n_estimators=1000,max_samples=100, bootstrap=True, n_jobs=-1, random_state=42).fit(xTrain, yTrain)
project_2.append({'Model':'Bagging Logistic','Train Score':bag_lg_clf.score(xTrain,yTrain),'Test Score':bag_lg_clf.score(xTest,yTest)})

bag_knn_clf = BaggingClassifier(knn, n_estimators=1000,max_samples=100, bootstrap=True, n_jobs=-1, random_state=42).fit(xTrain, yTrain)
project_2.append({'Model':'Bagging knn','Train Score':bag_knn_clf.score(xTrain,yTrain),'Test Score':bag_knn_clf.score(xTest,yTest)})

bagging= pd.DataFrame(project_2)
bagging

,Model,Train Score,Test Score
0,Ensembel hard,0.701357,0.655076
1,Ensembel soft,0.846850,0.678720
2,Bagging Logistic,0.669683,0.632823
3,Bagging knn,0.670031,0.632823


In [ ]:
#pasting
pst_dt_clf = BaggingClassifier(dt, n_estimators=1000,max_samples=100, bootstrap=False, n_jobs=-1, random_state=42).fit(xTrain, yTrain)
project_2.append({'Model':'Pasting Decision Tree','Train Score':pst_dt_clf.score(xTrain,yTrain),'Test Score':pst_dt_clf.score(xTest,yTest)})

pst_svcl_clf = BaggingClassifier(svcl, n_estimators=1000,max_samples=100, bootstrap=False, n_jobs=-1, random_state=42).fit(xTrain, yTrain)
project_2.append({'Model':'pasting Linear Svc','Train Score':pst_svcl_clf.score(xTrain,yTrain),'Test Score':pst_svcl_clf.score(xTest,yTest)})

pasting = pd.DataFrame(project_2)
pasting

,Model,Train Score,Test Score
0,Ensembel hard,0.701357,0.655076
1,Ensembel soft,0.846850,0.678720
2,Bagging Logistic,0.669683,0.632823
3,Bagging knn,0.670031,0.632823
4,Pasting Decision Tree,0.681170,0.642559
5,pasting Linear Svc,0.670031,0.632823


In [ ]:
#ADABoost

ada_rf_clf = AdaBoostClassifier(rf, n_estimators=1000,algorithm="SAMME.R", learning_rate=1, random_state=42).fit(xTrain, yTrain)
project_2.append({'Model':'ADABoost Random Forrest','Train Score':ada_rf_clf.score(xTrain,yTrain),'Test Score':ada_rf_clf.score(xTest,yTest)})
ada_dt_clf = AdaBoostClassifier(dt, n_estimators=1000,algorithm="SAMME.R", learning_rate=1, random_state=42).fit(xTrain, yTrain)
project_2.append({'Model':'ADABoost Decision Tree','Train Score':ada_dt_clf.score(xTrain,yTrain),'Test Score':ada_dt_clf.score(xTest,yTest)})

In [ ]:
min_samples_split=[20,40,80]
min_samples_leaf=[20,30]
max_depth=[4,5]
max_features=[12,13]
learning_rate=[0.01,0.1,1]
n_estimators=[100,500,1000]
params={
    'learning_rate':learning_rate,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'max_depth':max_depth,
    'max_features':max_features,
    'n_estimators':n_estimators}
gb_clf= GridSearchCV(GradientBoostingClassifier(),params).fit(xTrain,yTrain)
project_2.append({'Model':'Gradient Boost','Train Score':gb_clf.score(xTrain,yTrain),'Test Score':gb_clf.score(xTest,yTest)})
all_clf=pd.DataFrame(project_2)
all_clf

,Model,Train Score,Test Score
0,Ensembel hard,0.701357,0.655076
1,Ensembel soft,0.846850,0.678720
2,Bagging Logistic,0.669683,0.632823
3,Bagging knn,0.670031,0.632823
4,Pasting Decision Tree,0.681170,0.642559
5,pasting Linear Svc,0.670031,0.632823
6,ADABoost Random Forrest,0.995127,0.668985
7,ADABoost Decision Tree,1.000000,0.649513
8,Gradient Boost,0.832579,0.662031


In [ ]:
from sklearn.decomposition import PCA
pca=PCA(0.95)
pca.fit(xTrain)
xTrain_p=pca.transform(xTrain)
xTest_p=pca.transform(xTest)

In [ ]:
model2_scores=[]
#Logistic Regression
param={'C':[0.1,1,10],'penalty':['l1','l2']}
lg_clf= GridSearchCV(LogisticRegression(),param, cv=5).fit(xTrain_p,yTrain)
model2_scores.append({'Model':'LogisticRegression','Train Score':lg_clf.best_score_,'Test Score':lg_clf.score(xTest_p,yTest),'Parameters':lg_clf.best_params_})

#Decision Tree 
param={'max_depth': np.arange(3, 10)}
dt_clf=GridSearchCV(DecisionTreeClassifier(), param,cv=5).fit(xTrain_p,yTrain)
model2_scores.append({'Model':'Decision Tree','Train Score':dt_clf.best_score_,'Test Score':dt_clf.score(xTest_p,yTest),'Parameters':dt_clf.best_params_})

#KNN
param= {'n_neighbors':list(range(1,16))}
knn_clf= GridSearchCV(KNeighborsClassifier(),param,cv=5).fit(xTrain_p,yTrain)
model2_scores.append({'Model':'KNN','Train Score':knn_clf.best_score_,'Test Score':knn_clf.score(xTest_p,yTest),'Parameters':knn_clf.best_params_})

#Support Vector machine
Cs = [0.01, 0.1, 1, 10]
gammas = [0.01, 0.1, 1]
param_grid = {'C': Cs,'gamma':gammas}
#svc rbf
svcr_clf = GridSearchCV(SVC(kernel='rbf',probability=True), param_grid, cv=3).fit(xTrain_p,yTrain)
model2_scores.append({'Model':'SVC rbf','Train Score':svcr_clf.best_score_,'Test Score':svcr_clf.score(xTest_p,yTest),'Parameters':svcr_clf.best_params_})
#svc sigmoid
svcs_clf = GridSearchCV(SVC(kernel='sigmoid',probability=True), param_grid, cv=3).fit(xTrain_p,yTrain)
model2_scores.append({'Model':'SVC sigmoid','Train Score':svcs_clf.best_score_,'Test Score':svcs_clf.score(xTest_p,yTest),'Parameters':svcs_clf.best_params_})
#svc poly
svcp_clf = GridSearchCV(SVC(kernel='poly',probability=True), param_grid, cv=3).fit(xTrain_p,yTrain)
model2_scores.append({'Model':'SVC poly','Train Score':svcp_clf.best_score_,'Test Score':svcp_clf.score(xTest_p,yTest),'Parameters':svcp_clf.best_params_})
#Linear SVC
svcl_clf=GridSearchCV(SVC(kernel='linear',probability=True),param_grid={'C':Cs},cv=3).fit(xTrain_p,yTrain)
model2_scores.append({'Model':'SVC Linear','Train Score':svcl_clf.best_score_,'Test Score':svcl_clf.score(xTest_p,yTest),'Parameters':svcl_clf.best_params_})

#Random Forrest
param_grid = {
    'bootstrap': [True],
    'max_depth': [4,16,64],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [200, 300, 1000]
}
rf_clf = GridSearchCV(RandomForestClassifier(), param_grid = param_grid, n_jobs = -1, verbose = 2).fit(xTrain_p,yTrain)
model2_scores.append({'Model':'Random Forrest','Train Score':rf_clf.best_score_,'Test Score':rf_clf.score(xTest_p,yTest),'Parameters':rf_clf.best_params_})
project_1_pca= pd.DataFrame(model2_scores)
project_1_pca

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 14.5min finished


,Model,Train Score,Test Score,Parameters
0,LogisticRegression,0.685346,0.674548,"{'C': 0.1, 'penalty': 'l1'}"
1,Decision Tree,0.645667,0.618915,{'max_depth': 3}
2,KNN,0.654368,0.613352,{'n_neighbors': 9}
3,SVC rbf,0.679081,0.656467,"{'C': 1, 'gamma': 0.01}"
4,SVC sigmoid,0.671772,0.624478,"{'C': 0.01, 'gamma': 1}"
5,SVC poly,0.670031,0.632823,"{'C': 0.01, 'gamma': 0.01}"
6,SVC Linear,0.683258,0.645341,{'C': 10}
7,Random Forrest,0.677341,0.645341,"{'bootstrap': True, 'max_depth': 64, 'min_samp..."


In [ ]:
project_1.drop(['Parameters','Train Score'],axis=1).merge(project_1_pca.drop(['Parameters','Train Score'],axis=1), on=['Model'])

,Model,Test Score_x,Test Score_y
0,LogisticRegression,0.687065,0.674548
1,Decision Tree,0.632823,0.618915
2,KNN,0.621697,0.613352
3,SVC rbf,0.657858,0.656467
4,SVC sigmoid,0.630042,0.624478
5,SVC poly,0.632823,0.632823
6,SVC Linear,0.667594,0.645341
7,Random Forrest,0.655076,0.645341


PCA does improve the accuracy slightly, Logistic Regression still performs better than all the other model

In [ ]:
xTrain_p.shape

(2873, 75)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def create_model():
  model=Sequential()
  model.add(Dense(75, input_dim=75,activation='relu'))
  model.add(Dense(38,activation='relu'))
  model.add(Dense(18,activation='relu'))
  model.add(Dense(9,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics= ['accuracy'])

  return model

In [ ]:
seed= 10
np.random.seed(10)
model= KerasClassifier(build_fn=create_model,verbose=0)
param_grid= {'batch_size':[20,30], 'epochs': [50,150,300] }
nn_clf= GridSearchCV(model, param_grid=param_grid, cv=3 ).fit(xTrain_p,yTrain)

In [ ]:
print(nn_clf.score(xTrain_p,yTrain), nn_clf.score(xTest_p,yTest))

0.9996519314881779 0.6091794251400837
